In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.5 MB/s eta 0:00:00


In [3]:
import transformers
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import tensorflow as tf
import pandas as pd
import string
import re

import matplotlib.pyplot as plt
import seaborn as sns
from transformers import BertForSequenceClassification

from transformers import TFBertModel
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
import ast

import tensorflow_hub as hub


In [4]:
train_df = pd.read_csv("/content/drive/MyDrive/PFA/DATASET/trainset.csv",encoding='ISO-8859-1')
valid_df = pd.read_csv("/content/drive/MyDrive/PFA/DATASET/validationset.csv",encoding='ISO-8859-1')
test_df =pd.read_csv("/content/drive/MyDrive/PFA/DATASET/testset.csv",encoding='ISO-8859-1')

In [5]:
def concat_elements(text):
    elements = ast.literal_eval(text)  # Transform the string into a list
    concatenated = ' '.join(elements)  # Concatenate the elements of the list
    return concatenated

def conc_extract(df):
    df['all_cap'] = df['text_clean'].apply(concat_elements)
    return df[['all_cap', 'I-E', 'S-N', 'T-F', 'J-P']]  # Enclose column names in a list

In [6]:
train_df = conc_extract(train_df)
valid_df = conc_extract(valid_df)
test_df = conc_extract(test_df)

In [7]:
def transform_attr(value):
    if value == 1:
        return [1, 0]
    elif value == 0:
        return [0, 1]
    else:
        return []
y_train = train_df['I-E'].apply(lambda x: transform_attr(x))
y_train = np.array(y_train.to_list())

y_valid = valid_df['I-E'].apply(lambda x: transform_attr(x))
y_valid = np.array(y_valid.to_list())

y_test = train_df['I-E'].apply(lambda x: transform_attr(x))
y_test = np.array(y_test.to_list())

In [8]:
m_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(m_url, trainable=True)

In [9]:
tokenizer = transformers.AutoTokenizer.from_pretrained('bert-base-uncased',do_lower_case =True)

In [10]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
        
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len-len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
        
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [11]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def build_model(bert_layer, max_len=512):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
    
    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    
    clf_output = sequence_output[:, 0, :]
    
    lay = tf.keras.layers.Dense(64, activation='relu')(clf_output)
    lay = tf.keras.layers.Dropout(0.2)(lay)
    lay = tf.keras.layers.Dense(32, activation='relu')(lay)
    lay = tf.keras.layers.Dropout(0.2)(lay)
    out = tf.keras.layers.Dense(2, activation='sigmoid')(lay)
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=2e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model
    

In [12]:
max_len = 512
train_input = bert_encode(train_df.all_cap.values, tokenizer, max_len=max_len)
test_input = bert_encode(test_df.all_cap.values, tokenizer, max_len=max_len)
train_labels = y_train


Token indices sequence length is longer than the specified maximum sequence length for this model (1307 > 512). Running this sequence through the model will result in indexing errors


In [13]:
model = build_model(bert_layer, max_len=max_len)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 512)]        0           []                               
                                                                                                  
 segment_ids (InputLayer)       [(None, 512)]        0           []                               
                                                                                                  
 keras_layer (KerasLayer)       [(None, 768),        109482241   ['input_word_ids[0][0]',         
                                 (None, 512, 768)]                'input_mask[0][0]',         

In [14]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

train_sh = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=5,
    callbacks=[checkpoint, earlystopping],
    batch_size=32,
    verbose=1
)

Epoch 1/5


ResourceExhaustedError: ignored

In [ ]:
y_pred = model.predict(test_input)
print(y_pred)

In [ ]:
model.save('model_ie.h5')

In [ ]:
o